## Finales Modell
Nach Grid- und Random-Search werden die Ergebnisse der verschiedenen Modelle miteinander verglichen. Das beste Modell war eines aus der Grid-Search. Die Parameter werden übernommen und in das Modell eingepflegt. Anschließend wirt das Modell erstmalig auf die Testdaten losgelassen

In [1]:
import os
import numpy as np
import shutil
import tensorflow as tf
from typing import Tuple

from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

from tf_utils.dogscatsDataAdvanced import DOGSCATS

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/DOGSCATS/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [4]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    dense_layer_size: int,
    kernel_initializer: tf.keras.initializers.Initializer
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(input_img)
    x = Activation('relu')(x)
    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(
        units=dense_layer_size, 
        kernel_initializer=kernel_initializer
    )(x)
    x = Activation('relu')(x)
    x = Dense(
        units=num_classes, 
        kernel_initializer=kernel_initializer
    )(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [5]:
data = DOGSCATS()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

img_shape = data.img_shape
num_classes = data.num_classes

epochs = 40
batch_size = 128

# Hier nun die Parameter des Models der GridSearch eintragen
# param_grid={...} wird hier nicht mehr benötigt
# Best model params
optimizer = Adam
learning_rate = 0.001
filter_block1 = 32
kernel_size_block1 = 3
filter_block2 = 64
kernel_size_block2 = 3
filter_block3 = 128
kernel_size_block3 = 3
dense_layer_size = 128

In [11]:
# In diesem Video bzw. genau diesem Abschnitt werden verschiedene Kernel getestet
kernel_initializers = [
    # "GlorotUniform",
    "GlorotNormal",
    # "HeUniform",
    "HeNormal",
    # "LecunUniform",
    # "LecunNormal"
]

for kernel_initializer in kernel_initializers:
    model = build_model(
        img_shape,
        num_classes,
        optimizer,
        learning_rate,
        filter_block1,
        kernel_size_block1,
        filter_block2,
        kernel_size_block2,
        filter_block3,
        kernel_size_block3,
        dense_layer_size,
        kernel_initializer
    )
    model_log_dir = os.path.join(LOGS_DIR, f"model_{kernel_initializer}")

    tb_callback = TensorBoard(
        log_dir=model_log_dir,
        histogram_freq=0,
        profile_batch=0,
        write_graph=0
    )

    model.fit(
        train_dataset,
        verbose=1,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[tb_callback],
        validation_data=val_dataset,
    )

Epoch 1/40
105/105 [==============================] - 9s 68ms/step - loss: 0.7312 - accuracy: 0.5217 - val_loss: 0.6812 - val_accuracy: 0.5348
Epoch 2/40
105/105 [==============================] - 7s 63ms/step - loss: 0.6635 - accuracy: 0.5948 - val_loss: 0.6449 - val_accuracy: 0.6304
Epoch 3/40
105/105 [==============================] - 6s 62ms/step - loss: 0.6206 - accuracy: 0.6558 - val_loss: 0.5995 - val_accuracy: 0.6808
Epoch 4/40
105/105 [==============================] - 6s 61ms/step - loss: 0.5697 - accuracy: 0.7034 - val_loss: 0.5694 - val_accuracy: 0.7101
Epoch 5/40
105/105 [==============================] - 6s 61ms/step - loss: 0.5197 - accuracy: 0.7440 - val_loss: 0.4845 - val_accuracy: 0.7722
Epoch 6/40
105/105 [==============================] - 6s 61ms/step - loss: 0.4793 - accuracy: 0.7710 - val_loss: 0.5016 - val_accuracy: 0.7487
Epoch 7/40
105/105 [==============================] - 6s 60ms/step - loss: 0.4367 - accuracy: 0.7951 - val_loss: 0.4354 - val_accuracy: 0.7935

In [ ]:
print(f"Test performance best model: {val_accuracy}")

Test performance best model: [0.9349769949913025, 0.8416833877563477]
